In [15]:
from src.helpers import *
from sklearn.pipeline import Pipeline
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
import time
from umap import UMAP

In [16]:
# Reload the saved scaled data
X_train = pd.read_csv('data/train/scaled_X_train.csv', parse_dates=True, index_col='unix')
y_train = pd.read_csv('data/train/y_train.csv', parse_dates=True, index_col='unix')
X_test = pd.read_csv('data/test/scaled_X_test.csv', parse_dates=True, index_col='unix')
y_test = pd.read_csv('data/test/y_test.csv', parse_dates=True, index_col='unix')
X_val = pd.read_csv('data/val/scaled_X_val.csv', parse_dates=True, index_col='unix')
y_val = pd.read_csv('data/val/y_val.csv', parse_dates=True, index_col='unix')

In [17]:
start = time.time()
for i in range(1, 1000):
    continue
end = time.time()
duration = '{0:.5f}'.format(end - start)
print(f'{duration} seconds')

0.00005 seconds


In [18]:
# Calculate the weights for the imbalanced classes
y = pd.concat([y_train, y_val, y_test])
weights = cwts(y.values.flatten())


In [19]:
rf = RandomForestClassifier(n_jobs=-1, class_weight=weights)

In [20]:
# Define the various feature selection methods
vif = VIFTransform(dropna=True)
boruta = BorutaPy(rf, n_estimators='auto', verbose=2)

In [21]:
# Define data pipelines
pipelines = [
    'none',
    Pipeline([('vif', vif)])
]

In [22]:
umap = UMAP(n_neighbors=5)
pipe_1 = Pipeline([('umap', umap)])

In [23]:
filepath = './tensorboard/feature_selection/run'
time = datetime.now().strftime('%m-%d-%Y-%H:%M:%S')
for key in pipe_1.named_steps.keys():
    filepath += f'_{key}_'
filepath += time
filepath

'./tensorboard/feature_selection/run_umap_01-01-2024-16:50:00'

In [24]:
X_train = pipe_1.fit_transform(X_train, y_train.values.ravel())

In [26]:
X_train

array([[13.877993 ,  8.688454 ],
       [13.876389 ,  8.687096 ],
       [13.508537 ,  8.250653 ],
       ...,
       [ 6.748612 , 11.521355 ],
       [ 6.758693 , 11.0943575],
       [ 6.2896395, 11.115365 ]], dtype=float32)